In [1]:
import jcopdl
import luwiji

jcopdl.__version__, luwiji.__version__

('2.3.3', '1.2.6')

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback
from jcopdl.callback import set_config

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# Dataset & Dataloader

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [7]:
bs = 128 # batch_size | standarnya 128
crop_size = 64 # ukuran crop | standarnya 224

train_transform = transforms.Compose([
    # augmentasi data :
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(crop_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),

    # mengubah ke tipe data tensor :
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    # data test tidak perlu diaugmentasi
    # di-resize dulu, baru di-crop
    transforms.Resize(70),
    transforms.CenterCrop(crop_size),

    # mengubah ke tipe data tensor :
    transforms.ToTensor()
])

train_set = datasets.ImageFolder('data/train', transform=train_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=4)

test_set = datasets.ImageFolder('data/test', transform=test_transform)
testloader = DataLoader(test_set, batch_size=bs, shuffle='')

In [8]:
feature, target = next(iter(trainloader))
feature.shape

torch.Size([128, 3, 64, 64])

In [9]:
label2cat = train_set.classes
label2cat

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Arsitektur & Config

In [12]:
from jcopdl.layers import conv_block, linear_block

# jika manual dan tanpa blok :
# nn.Conv2d(3, 8, 3, 1, 1)
# nn.ReLU(),
# nn.MaxPool2d(2, 2)

In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(
            conv_block(3, 8),
            conv_block(8, 16),
            conv_block(16, 32),
            conv_block(32, 64),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            linear_block(1024, 256, dropout=0.1),
            linear_block(256, 2, activation="lsoftmax")
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [ ]:
config = set_config({
    'batch_size': bs,
    'crop_size': crop_size
})

# Training Preparation -> MCOC

In [ ]:
model = CNN.to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config, outdir='model')

# Training

In [ ]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == 'train':
        model.train()
    elif mode == 'test':
        model.eval()
    const = correct = 0
    
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)

        if mode == 'train':
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [ ]:
while True:
    train_cost, train_score = loop_fn('train', train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn('test', test_set, testloader, model, criterion, optimizer, device)
    
    # Logging
    callback.log(train_cost, test_cost, train_score, test_score)

    # Checkpoint
    callback.save_checkpoint()

    # Runtime Plotting
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()

    # Early Stopping
    if callback.early_stopping(model, monitor='test_score'):
        callback.plot_cost()
        callback.plot_score()
        break